# Automated Prescription From Patient Texts Initial Notebook
## Rough Modelling

In [2]:
# https://github.com/dreji18/Bio-Epidemiology-NER

## We will get to the actual work now

In [3]:
import pandas as pd
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

import torch
import torch.optim as optim
import torch.nn as nn

In [4]:
# reading disease prediction dataset 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

dis_df = pd.read_csv("/kaggle/input/disease-prediction-dataset/disease_prediction_datasets/Training.csv")
dis_df = dis_df.rename(columns={'fluid_overload.1': 'fluid_overload', 'toxic_look_(typhos)': 'toxic_look_typhos' , 'spotting_ urination' : 'spotting_urination', 'dischromic _patches' : 'dischromic_patches', 'foul_smell_of urine' : 'foul_smell_of_urine'})
dis_df = dis_df.drop('Unnamed: 133', axis=1)
dis_df = dis_df.loc[:,~dis_df.columns.duplicated()]

dis_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection


In [5]:
dis_df.shape

(4920, 132)

## We need to add more data to this dataset
We may use this dataset : https://www.kaggle.com/datasets/uom190346a/disease-symptoms-and-patient-profile-dataset


In [6]:
dis_2 = pd.read_csv("/kaggle/input/more-diseases/Disease_symptom_and_patient_profile_dataset.csv")
dis_2.shape

(349, 10)

In [7]:
dis_2.dtypes

Disease                 object
Fever                   object
Cough                   object
Fatigue                 object
Difficulty Breathing    object
Age                      int64
Gender                  object
Blood Pressure          object
Cholesterol Level       object
Outcome Variable        object
dtype: object

In [8]:
dis_2.head()

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [9]:
dis_2['Cholesterol Level'].value_counts()

Cholesterol Level
High      166
Normal    149
Low        34
Name: count, dtype: int64

### DAta Preprocessing for Symptom Prediction

In [10]:
dis_df_cp = dis_df.copy()


In [11]:
dis_df_cp.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection


In [12]:
dis_2.columns = dis_2.columns.str.lower()

# Filter rows where Outcome Variable is Positive
dis_2 = dis_2[dis_2['outcome variable'] == 'Positive']
dis_2 = dis_2.drop('outcome variable', axis=1)

In [13]:
dis_2 = dis_2.rename(columns={'disease': 'prognosis'})

In [14]:
dis_2 = dis_2.drop(['age', 'gender'], axis=1)

In [15]:
dis_2.head()

,prognosis,fever,cough,fatigue,difficulty breathing,blood pressure,cholesterol level
0,Influenza,Yes,No,Yes,Yes,Low,Normal
3,Asthma,Yes,Yes,No,Yes,Normal,Normal
4,Asthma,Yes,Yes,No,Yes,Normal,Normal
5,Eczema,Yes,No,No,No,Normal,Normal
6,Influenza,Yes,Yes,Yes,Yes,Normal,Normal


In [16]:
# duplicate fever column and rename fever = high_fever, another one mild_fever 
dis_2['high_fever'] = dis_2['fever'].copy()
dis_2 = dis_2.rename(columns={'fever': 'mild_fever'})


In [17]:
# make difficulty breathing = difficulty_breathing and blood pressure = blood_pressure
dis_2 = dis_2.rename(columns={
    'difficulty breathing': 'difficulty_breathing',
    'blood pressure': 'blood_pressure',
    'cholesterol level': 'cholesterol_level'
})

In [18]:
dis_2.head()

,prognosis,mild_fever,cough,fatigue,difficulty_breathing,blood_pressure,cholesterol_level,high_fever
0,Influenza,Yes,No,Yes,Yes,Low,Normal,Yes
3,Asthma,Yes,Yes,No,Yes,Normal,Normal,Yes
4,Asthma,Yes,Yes,No,Yes,Normal,Normal,Yes
5,Eczema,Yes,No,No,No,Normal,Normal,Yes
6,Influenza,Yes,Yes,Yes,Yes,Normal,Normal,Yes


In [19]:
dis_2['prognosis'].nunique()

77

In [20]:
# Convert yes/high = 1, no/low/normal = 0
columns_to_convert = ['mild_fever', 'cough', 'fatigue', 'difficulty_breathing', 'high_fever']
dis_2[columns_to_convert] = dis_2[columns_to_convert].replace({'Yes': 1, 'No': 0})

columns_to_convert = ['blood_pressure', 'cholesterol_level']
dis_2[columns_to_convert] = dis_2[columns_to_convert].replace({'High': 1, 'Low': 0, 'Normal': 0})

/tmp/ipykernel_33/2939065223.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dis_2[columns_to_convert] = dis_2[columns_to_convert].replace({'Yes': 1, 'No': 0})
/tmp/ipykernel_33/2939065223.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dis_2[columns_to_convert] = dis_2[columns_to_convert].replace({'High': 1, 'Low': 0, 'Normal': 0})


In [21]:
# add this data to the dis_df_cp , for other remainig columns fill with 0
dis_2.head()


,prognosis,mild_fever,cough,fatigue,difficulty_breathing,blood_pressure,cholesterol_level,high_fever
0,Influenza,1,0,1,1,0,0,1
3,Asthma,1,1,0,1,0,0,1
4,Asthma,1,1,0,1,0,0,1
5,Eczema,1,0,0,0,0,0,1
6,Influenza,1,1,1,1,0,0,1


In [22]:
dis_df_cp.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection


In [23]:
# Find columns in dis_2 that are not in dis_df_cp
extra_columns_in_dis_2 = [col for col in dis_2.columns if col not in dis_df_cp.columns]

# Display the extra columns
print("Columns in dis_2 that are not in dis_df_cp:")
print(extra_columns_in_dis_2)

Columns in dis_2 that are not in dis_df_cp:
['difficulty_breathing', 'blood_pressure', 'cholesterol_level']


In [24]:
# Create a DataFrame with the extra columns initialized to NaN for existing rows in dis_df_cp
extra_columns_df = pd.DataFrame(pd.NA, index=dis_df_cp.index, columns=extra_columns_in_dis_2)

# Concatenate this new DataFrame with dis_df_cp to include the extra columns
dis_df_cp_with_extras = pd.concat([dis_df_cp, extra_columns_df], axis=1)


In [25]:
# Append rows from dis_2 to dis_df_cp_with_extras
final_dis_df = pd.concat([dis_df_cp_with_extras, dis_2], ignore_index=True)

In [26]:
final_dis_df.tail()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,difficulty_breathing,blood_pressure,cholesterol_level
5101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stroke,0,1,1
5102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stroke,0,1,1
5103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stroke,0,1,1
5104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [27]:
# Find columns with non-numeric types and their unique values in final_dis_df
non_numeric_columns = []
for col in final_dis_df.columns:
    if not pd.api.types.is_numeric_dtype(final_dis_df[col]):
        non_numeric_columns.append(col)
        unique_values = final_dis_df[col].unique()
        print(f"Column '{col}' has non-numeric type with unique values:")
        print(unique_values)
        print()

# Display non-numeric columns and their unique values
print("Non-numeric columns and their unique values:")
print(non_numeric_columns)

Column 'prognosis' has non-numeric type with unique values:
['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction'
 'Peptic ulcer diseae' 'AIDS' 'Diabetes ' 'Gastroenteritis'
 'Bronchial Asthma' 'Hypertension ' 'Migraine' 'Cervical spondylosis'
 'Paralysis (brain hemorrhage)' 'Jaundice' 'Malaria' 'Chicken pox'
 'Dengue' 'Typhoid' 'hepatitis A' 'Hepatitis B' 'Hepatitis C'
 'Hepatitis D' 'Hepatitis E' 'Alcoholic hepatitis' 'Tuberculosis'
 'Common Cold' 'Pneumonia' 'Dimorphic hemmorhoids(piles)' 'Heart attack'
 'Varicose veins' 'Hypothyroidism' 'Hyperthyroidism' 'Hypoglycemia'
 'Osteoarthristis' 'Arthritis' '(vertigo) Paroymsal  Positional Vertigo'
 'Acne' 'Urinary tract infection' 'Psoriasis' 'Impetigo' 'Influenza'
 'Asthma' 'Eczema' 'Depression' 'Liver Cancer' 'Stroke'
 'Urinary Tract Infection' 'Bipolar Disorder' 'Bronchitis'
 'Cerebral Palsy' 'Colorectal Cancer' 'Hypertensive Heart Disease'
 'Multiple Sclerosis' 'Myocardial Infarction (Heart...'
 'Urinary Tract Inf

In [28]:
# Fill NaN or <NA> values with 0
final_dis_df = final_dis_df.fillna(0)

/tmp/ipykernel_33/2981334920.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_dis_df = final_dis_df.fillna(0)


In [29]:
final_dis_df.dtypes

itching                           float64
skin_rash                         float64
nodal_skin_eruptions              float64
continuous_sneezing               float64
shivering                         float64
chills                            float64
joint_pain                        float64
stomach_pain                      float64
acidity                           float64
ulcers_on_tongue                  float64
muscle_wasting                    float64
vomiting                          float64
burning_micturition               float64
spotting_urination                float64
fatigue                             int64
weight_gain                       float64
anxiety                           float64
cold_hands_and_feets              float64
mood_swings                       float64
weight_loss                       float64
restlessness                      float64
lethargy                          float64
patches_in_throat                 float64
irregular_sugar_level             

In [30]:
# Convert float64 columns to integers
for col in final_dis_df.columns:
    if final_dis_df[col].dtype == 'float64':
        final_dis_df[col] = final_dis_df[col].astype(int)

In [31]:
final_dis_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,difficulty_breathing,blood_pressure,cholesterol_level
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,0,0,0
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,0,0,0
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection,0,0,0


In [32]:
final_dis_df.shape

(5106, 135)

# Instruction From Sir 
## DO WEIGHTED TRAINING ON DISEASE PREDICTION MODEL
## cheeck for correlation features , correlation map, check for balance, do data augmentation - oversampling, SMOTE

In [33]:
final_dis_df.shape

(5106, 135)

## Generating Keywords and Data for NER

In [34]:
#### add the keywords for new added columns

In [35]:
keywords_new = {
    "itching": ["itching", "itch"],
    "skin_rash": ["skin rash", "rash"],
    "nodal_skin_eruptions": ["nodal skin eruptions"],
    "continuous_sneezing": ["continuous sneezing"],
    "shivering": ["shivering", "chills"],
    "chills": ["chills", "fever"],
    "joint_pain": ["joint pain", "arthritis"],
    "stomach_pain": ["stomach pain", "abdominal pain"],
    "acidity": ["acidity", "heartburn"],
    "ulcers_on_tongue": ["ulcers on tongue", "mouth sores"],
    "muscle_wasting": ["muscle wasting", "muscle weakness"],
    "vomiting": ["vomiting", "nausea"],
    "burning_micturition": ["burning micturition", "painful urination"],
    "spotting_urination": ["spotting urination", "frequent urination"],
    "fatigue": ["fatigue", "tiredness"],
    "weight_gain": ["weight gain", "obesity"],
    "anxiety": ["anxiety", "stress"],
    "cold_hands_and_feets": ["cold hands and feet", "poor circulation"],
    "mood_swings": ["mood swings", "irritability"],
    "weight_loss": ["weight loss", "slimming"],
    "restlessness": ["restlessness", "insomnia"],
    "lethargy": ["lethargy", "apathy"],
    "patches_in_throat": ["patches in throat", "sore throat"],
    "irregular_sugar_level": ["irregular sugar level", "diabetes"],
    "cough": ["cough", "respiratory issues"],
    "high_fever": ["high fever", "temperature elevation"],
    "sunken_eyes": ["sunken eyes", "fatigue"],
    "breathlessness": ["breathlessness", "shortness of breath"],
    "sweating": ["sweating", "perspiration"],
    "dehydration": ["dehydration", "water loss"],
    "indigestion": ["indigestion", "heartburn"],
    "headache": ["headache", "migraine"],
    "yellowish_skin": ["yellowish skin", "jaundice"],
    "dark_urine": ["dark urine", "kidney issues"],
    "nausea": ["nausea", "vomiting"],
    "loss_of_appetite": ["loss of appetite", "decreased hunger"],
    "pain_behind_the_eyes": ["pain behind the eyes", "eye strain"],
    "back_pain": ["back pain", "muscle strain"],
    "constipation": ["constipation", "bowel issues"],
    "abdominal_pain": ["abdominal pain", "stomach cramps"],
    "diarrhoea": ["diarrhoea", "frequent bowel movements"],
    "mild_fever": ["mild fever", "low-grade temperature"],
    "yellow_urine": ["yellow urine", "urine color change"],
    "yellowing_of_eyes": ["yellowing of eyes", "jaundice"],
    "acute_liver_failure": ["acute liver failure", "liver dysfunction"],
    "fluid_overload": ["fluid overload", "edema"],
    "swelling_of_stomach": ["swelling of stomach", "abdominal distension"],
    "swelled_lymph_nodes": ["swelled lymph nodes", "enlarged lymph nodes"],
    "malaise": ["malaise", "general discomfort"],
    "blurred_and_distorted_vision": ["blurred and distorted vision", "eye problems"],
    "phlegm": ["phlegm", "mucus"],
    "throat_irritation": ["throat irritation", "sore throat"],
    "redness_of_eyes": ["redness of eyes", "eye irritation"],
    "sinus_pressure": ["sinus pressure", "congestion"],
    "runny_nose": ["runny nose", "rhinorrhea"],
    "congestion": ["congestion", "stuffy nose"],
    "chest_pain": ["chest pain", "cardiac issues"],
    "weakness_in_limbs": ["weakness in limbs", "muscle weakness"],
    "fast_heart_rate": ["fast heart rate", "tachycardia"],
    "pain_during_bowel_movements": ["pain during bowel movements", "rectal pain"],
    "pain_in_anal_region": ["pain in anal region", "anal pain"],
    "bloody_stool": ["bloody stool", "hematochezia"],
    "irritation_in_anus": ["irritation in anus", "anal itching"],
    "neck_pain": ["neck pain", "cervical pain"],
    "dizziness": ["dizziness", "lightheadedness"],
    "cramps": ["cramps", "muscle spasms"],
    "bruising": ["bruising", "ecchymosis"],
    "obesity": ["obesity", "overweight"],
    "swollen_legs": ["swollen legs", "edema"],
    "swollen_blood_vessels": ["swollen blood vessels", "vasculitis"],
    "puffy_face_and_eyes": ["puffy face and eyes", "edema"],
    "enlarged_thyroid": ["enlarged thyroid", "goiter"],
    "brittle_nails": ["brittle nails", "nail fragility"],
    "swollen_extremeties": ["swollen extremeties", "edema"],
    "excessive_hunger": ["excessive hunger", "polyphagia"],
    "extra_marital_contacts": ["extra marital contacts", "infidelity"],
    "drying_and_tingling_lips": ["drying and tingling lips", "cheilitis"],
    "slurred_speech": ["slurred speech", "dysarthria"],
    "knee_pain": ["knee pain", "knee joint pain"],
    "hip_joint_pain": ["hip joint pain", "hip arthritis"],
    "muscle_weakness": ["muscle weakness", "muscle atrophy"],
    "stiff_neck": ["stiff neck", "cervical stiffness"],
    "swelling_joints": ["swelling joints", "joint swelling"],
    "movement_stiffness": ["movement stiffness", "muscle rigidity"],
    "spinning_movements": ["spinning movements", "vertigo"],
    "loss_of_balance": ["loss of balance", "ataxia"],
    "unsteadiness": ["unsteadiness", "lightheadedness"],
    "weakness_of_one_body_side": ["weakness of one body side", "hemiparesis"],
    "loss_of_smell": ["loss of smell", "anosmia"],
    "bladder_discomfort": ["bladder discomfort", "urinary issues"],
    "foul_smell_of_urine": ["foul smell of urine", "urinary tract infection"],
    "continuous_feel_of_urine": ["continuous feel of urine", "urinary frequency"],
    "passage_of_gases": ["passage of gases", "flatulence"],
    "internal_itching": ["internal itching", "pruritus"],
    "toxic_look_typhos": ["toxic look typhos", "typhoid fever"],
    "depression": ["depression", "mental health disorders"],
    "irritability": ["irritability", "mood swings"],
    "muscle_pain": ["muscle pain", "myalgia"],
    "altered_sensorium": ["altered sensorium", "confusion"],
    "red_spots_over_body": ["red spots over body", "petechiae"],
    "belly_pain": ["belly pain", "abdominal pain"],
    "abnormal_menstruation": ["abnormal menstruation", "menstrual irregularities"],
    "dischromic_patches": ["dischromic patches", "skin discoloration"],
    "watering_from_eyes": ["watering from eyes", "lacrimation"],
    "increased_appetite": ["increased appetite", "polyphagia"],
    "polyuria": ["polyuria", "frequent urination"],
    "family_history": ["family history", "genetic disorders"],
    "mucoid_sputum": ["mucoid sputum", "respiratory issues"],
    "rusty_sputum": ["rusty sputum", "hemoptysis"],
    "lack_of_concentration": ["lack of concentration", "attention deficit"],
    "visual_disturbances": ["visual disturbances", "blurred vision"],
    "receiving_blood_transfusion": ["receiving blood transfusion", "blood transfusion"],
    "receiving_unsterile_injections": ["receiving unsterile injections", "unsterile injections"],
    "coma": ["coma", "unconsciousness"],
    "stomach_bleeding": ["stomach bleeding", "gastrointestinal bleeding"],
    "distention_of_abdomen": ["distention of abdomen", "abdominal swelling"],
    "history_of_alcohol_consumption": ["history of alcohol consumption", "alcoholism"],
    "fluid_overload": ["fluid overload", "edema"],
    "blood_in_sputum": ["blood in sputum", "hemoptysis"],
    "prominent_veins_on_calf": ["prominent veins on calf", "varicose veins"],
    "palpitations": ["palpitations", "heart palpitations"],
    "painful_walking": ["painful walking", "foot pain"],
    "pus_filled_pimples": ["pus filled pimples", "acne"],
    "blackheads": ["blackheads", "comedones"],
    "scurring": ["scurring", "scarring"],
    "skin_peeling": ["skin peeling", "exfoliation"],
    "silver_like_dusting": ["silver like dusting", "skin discoloration"],
    "small_dents_in_nails": ["small dents in nails", "nail abnormalities"],
    "inflammatory_nails": ["inflammatory nails", "nail inflammation"],
    "blister": ["blister", "fluid-filled bumps"],
    "red_sore_around_nose": ["red sore around nose", "nasal irritation"],
    "yellow_crust_ooze": ["yellow crust ooze", "pus-filled crust"],
    "difficulty_breathing": ["difficulty breathing", "shortness of breath", "breathlessness", "labored breathing", "dyspnea", "breath"],
    "blood_pressure": ["blood pressure", "hypertension", "high BP", "high blood pressure", "high pressure"],
    "cholesterol_level": ["cholesterol", "high cholesterol" , "high lipid levels" , "LDL"]
}

In [36]:
len(keywords_new)

134

In [37]:
execution break

SyntaxError: invalid syntax (1655976014.py, line 1)

In [38]:
import random

# Function to generate training data based on keywords
def generate_training_data(keywords, num_samples=1000):
    training_data = []
    for _ in range(num_samples):
        # Randomly select a symptom and generate a sentence containing it
        symptom = random.choice(list(keywords.keys()))
        keyword = random.choice(keywords[symptom])
        sentence = f"I have {keyword}."
        start = sentence.find(keyword)
        end = start + len(keyword)
        # Annotate the sentence with the symptom entity
        annotations = {"entities": [(start, end, symptom.upper())]}
        training_data.append((sentence, annotations))
    return training_data

# Generate training data
training_data = generate_training_data(keywords_new, num_samples=1000)

# Print sample training data
for text, annotations in training_data[:5]:
    print("Text:", text)
    print("Annotations:", annotations)
    print()


Text: I have infidelity.
Annotations: {'entities': [(7, 17, 'EXTRA_MARITAL_CONTACTS')]}

Text: I have breathlessness.
Annotations: {'entities': [(7, 21, 'BREATHLESSNESS')]}

Text: I have swelling joints.
Annotations: {'entities': [(7, 22, 'SWELLING_JOINTS')]}

Text: I have skin rash.
Annotations: {'entities': [(7, 16, 'SKIN_RASH')]}

Text: I have goiter.
Annotations: {'entities': [(7, 13, 'ENLARGED_THYROID')]}



In [39]:
ner_data = pd.DataFrame(training_data, columns=["text", "annotations"])
ner_data.head()

,text,annotations
0,I have infidelity.,"{'entities': [(7, 17, 'EXTRA_MARITAL_CONTACTS')]}"
1,I have breathlessness.,"{'entities': [(7, 21, 'BREATHLESSNESS')]}"
2,I have swelling joints.,"{'entities': [(7, 22, 'SWELLING_JOINTS')]}"
3,I have skin rash.,"{'entities': [(7, 16, 'SKIN_RASH')]}"
4,I have goiter.,"{'entities': [(7, 13, 'ENLARGED_THYROID')]}"


## Symptom Tracker NER Modelling

In [40]:
# !python -m spacy download en_core_web_sm

In [41]:
# # Load a blank spaCy model
# nlp = spacy.blank("en")

# # Create the NER component and add it to the pipeline
# if "ner" not in nlp.pipe_names:
#     ner = nlp.add_pipe("ner")
# else:
#     ner = nlp.get_pipe("ner")

# # Add labels to the NER component
# for _, annotations in training_data:
#     for ent in annotations.get("entities"):
#         ner.add_label(ent[2])

# # Convert the data to spaCy's format
# train_data = []
# for text, annotations in training_data:
#     train_data.append(Example.from_dict(nlp.make_doc(text), annotations))

# # Train the NER model
# optimizer = nlp.begin_training()
# for itn in range(300):
#     losses = {}
#     batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
#     for batch in batches:
#         nlp.update(batch, drop=0.5, losses=losses)
#     print(f"Losses at iteration {itn}: {losses}")

# # Save the model
# nlp.to_disk("disease_ner_model")

In [42]:
# Load the trained model
nlp = spacy.load("/kaggle/input/disease-ner-model/disease_ner_model")

# Function to map entities to prediction model columns
def map_entities_to_columns(text, symptom_keywords):
    # Initialize mapping with zeros for all symptoms
    mapping = {symptom: 0 for symptom in symptom_keywords.keys()}
    
    # Process text with the NER model
    doc = nlp(text)
    
    # Update mapping based on detected entities
    for ent in doc.ents:
        symptom = ent.label_.lower()
        if symptom in mapping:
            mapping[symptom] = 1
            
    return mapping


In [43]:
import re

# Function to map keywords to prediction model columns
def map_keywords_to_columns(text, symptom_keywords):
    # Initialize mapping with zeros for all symptoms
    mapping = {symptom: 0 for symptom in symptom_keywords.keys()}
    
    # Check for keyword matches in the text
    for symptom, keys in symptom_keywords.items():
        pattern = re.compile(r'\b(' + '|'.join(keys) + r')\b', re.IGNORECASE)
        matches = pattern.findall(text)
        if matches:
            mapping[symptom] = 1
            
    return mapping

In [44]:
text = "I have itching and knee pain."

result = map_keywords_to_columns(text, keywords_new)

print(result)

{'itching': 1, 'skin_rash': 0, 'nodal_skin_eruptions': 0, 'continuous_sneezing': 0, 'shivering': 0, 'chills': 0, 'joint_pain': 0, 'stomach_pain': 0, 'acidity': 0, 'ulcers_on_tongue': 0, 'muscle_wasting': 0, 'vomiting': 0, 'burning_micturition': 0, 'spotting_urination': 0, 'fatigue': 0, 'weight_gain': 0, 'anxiety': 0, 'cold_hands_and_feets': 0, 'mood_swings': 0, 'weight_loss': 0, 'restlessness': 0, 'lethargy': 0, 'patches_in_throat': 0, 'irregular_sugar_level': 0, 'cough': 0, 'high_fever': 0, 'sunken_eyes': 0, 'breathlessness': 0, 'sweating': 0, 'dehydration': 0, 'indigestion': 0, 'headache': 0, 'yellowish_skin': 0, 'dark_urine': 0, 'nausea': 0, 'loss_of_appetite': 0, 'pain_behind_the_eyes': 0, 'back_pain': 0, 'constipation': 0, 'abdominal_pain': 0, 'diarrhoea': 0, 'mild_fever': 0, 'yellow_urine': 0, 'yellowing_of_eyes': 0, 'acute_liver_failure': 0, 'fluid_overload': 0, 'swelling_of_stomach': 0, 'swelled_lymph_nodes': 0, 'malaise': 0, 'blurred_and_distorted_vision': 0, 'phlegm': 0, 'thr

In [45]:
len(result)

134

In [46]:
# Example usage
text = "I have itching and knee pain."

result = map_entities_to_columns(text, keywords_new)
print(result)

{'itching': 1, 'skin_rash': 0, 'nodal_skin_eruptions': 0, 'continuous_sneezing': 0, 'shivering': 1, 'chills': 0, 'joint_pain': 0, 'stomach_pain': 0, 'acidity': 0, 'ulcers_on_tongue': 0, 'muscle_wasting': 0, 'vomiting': 0, 'burning_micturition': 0, 'spotting_urination': 0, 'fatigue': 0, 'weight_gain': 0, 'anxiety': 0, 'cold_hands_and_feets': 0, 'mood_swings': 0, 'weight_loss': 0, 'restlessness': 0, 'lethargy': 0, 'patches_in_throat': 0, 'irregular_sugar_level': 0, 'cough': 0, 'high_fever': 0, 'sunken_eyes': 0, 'breathlessness': 0, 'sweating': 0, 'dehydration': 0, 'indigestion': 0, 'headache': 0, 'yellowish_skin': 0, 'dark_urine': 0, 'nausea': 0, 'loss_of_appetite': 0, 'pain_behind_the_eyes': 0, 'back_pain': 0, 'constipation': 0, 'abdominal_pain': 0, 'diarrhoea': 0, 'mild_fever': 0, 'yellow_urine': 0, 'yellowing_of_eyes': 0, 'acute_liver_failure': 0, 'fluid_overload': 0, 'swelling_of_stomach': 0, 'swelled_lymph_nodes': 0, 'malaise': 0, 'blurred_and_distorted_vision': 0, 'phlegm': 0, 'thr

In [47]:
#### Combinign keyword and NER model

def map_symptoms(text):
    # First, use NER to map entities
    mapping = map_entities_to_columns(text, keywords_new)
    
    # Then, use refined keyword-based mapping as a fallback
    keyword_mapping = map_keywords_to_columns(text, keywords_new)
    
    # Update the mapping only if the NER did not find the symptom
    for symptom in mapping.keys():
        if mapping[symptom] == 0 and keyword_mapping[symptom] == 1:
            mapping[symptom] = 1
    return mapping

In [48]:
# Example usage with combined mapping
test_text = "I am 22 years old. I am facing nose itching and vomiting with i have a severe chest pain and high fever."
mapped_columns = map_symptoms(test_text)
print(f"Combined mapping: {mapped_columns}")

Combined mapping: {'itching': 1, 'skin_rash': 0, 'nodal_skin_eruptions': 0, 'continuous_sneezing': 0, 'shivering': 1, 'chills': 1, 'joint_pain': 0, 'stomach_pain': 0, 'acidity': 0, 'ulcers_on_tongue': 0, 'muscle_wasting': 0, 'vomiting': 1, 'burning_micturition': 0, 'spotting_urination': 0, 'fatigue': 0, 'weight_gain': 0, 'anxiety': 0, 'cold_hands_and_feets': 0, 'mood_swings': 0, 'weight_loss': 0, 'restlessness': 0, 'lethargy': 0, 'patches_in_throat': 0, 'irregular_sugar_level': 0, 'cough': 0, 'high_fever': 1, 'sunken_eyes': 0, 'breathlessness': 0, 'sweating': 0, 'dehydration': 0, 'indigestion': 0, 'headache': 0, 'yellowish_skin': 1, 'dark_urine': 0, 'nausea': 1, 'loss_of_appetite': 0, 'pain_behind_the_eyes': 0, 'back_pain': 0, 'constipation': 0, 'abdominal_pain': 0, 'diarrhoea': 0, 'mild_fever': 0, 'yellow_urine': 0, 'yellowing_of_eyes': 0, 'acute_liver_failure': 1, 'fluid_overload': 0, 'swelling_of_stomach': 0, 'swelled_lymph_nodes': 0, 'malaise': 0, 'blurred_and_distorted_vision': 1,

In [49]:
len(mapped_columns)

134

## Disease Prediction Model
Now we will train the baseline disease prediction model and push the symptoms to the model for prediction

In [50]:
le = LabelEncoder()
final_dis_df['prognosis'] = le.fit_transform(final_dis_df['prognosis'])

In [51]:
X = final_dis_df.drop('prognosis', axis=1)
y = final_dis_df['prognosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
X.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,excessive_hunger,extra_marital_contacts,drying_and_tingling_lips,slurred_speech,knee_pain,hip_joint_pain,muscle_weakness,stiff_neck,swelling_joints,movement_stiffness,spinning_movements,loss_of_balance,unsteadiness,weakness_of_one_body_side,loss_of_smell,bladder_discomfort,foul_smell_of_urine,continuous_feel_of_urine,passage_of_gases,internal_itching,toxic_look_typhos,depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic_patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,difficulty_breathing,blood_pressure,cholesterol_level
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
X.shape

(5106, 134)

In [54]:
X.dtypes

itching                           int64
skin_rash                         int64
nodal_skin_eruptions              int64
continuous_sneezing               int64
shivering                         int64
chills                            int64
joint_pain                        int64
stomach_pain                      int64
acidity                           int64
ulcers_on_tongue                  int64
muscle_wasting                    int64
vomiting                          int64
burning_micturition               int64
spotting_urination                int64
fatigue                           int64
weight_gain                       int64
anxiety                           int64
cold_hands_and_feets              int64
mood_swings                       int64
weight_loss                       int64
restlessness                      int64
lethargy                          int64
patches_in_throat                 int64
irregular_sugar_level             int64
cough                             int64


In [55]:
len(mapped_columns.keys())

134

In [56]:
len(keywords_new.keys())

134

In [57]:
param_grid = {
    'max_depth': [5, 10, 15],
    'n_estimators': [50, 100, 200]
}

model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [58]:
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

model = grid_search.best_estimator_
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Best Parameters: {'max_depth': 10, 'n_estimators': 100}
Best Accuracy: 0.9640065879472965
Test Accuracy: 0.961839530332681


## Pushing the symptom data to disease prediction model to get the output


In [59]:
# Define the pipeline
pipeline = Pipeline([
    ('symptom_mapping', map_symptoms),
    ('create_dataframe', lambda x: pd.DataFrame([x], columns=x.keys())),
    ('disease_classification', model)
])

# Define a function to take text input and return the predicted disease
def predict_disease(text):
    symptoms = pipeline['symptom_mapping'](text)
    df = pipeline['create_dataframe'](symptoms)
    disease = pipeline['disease_classification'].predict(df)
    disease_name = le.inverse_transform(pd.Series(disease).to_numpy())[0]
    return disease_name


In [60]:
# Test the pipeline
text = "Doc, I've been feeling really crummy lately. I've had a nasty cough for weeks, and it's been getting worse. My chest hurts, and I feel like I can't catch my breath. I've also had a fever on and off, and I'm just so tired all the time. Sometimes I get these sharp pains in my head, and my nose is always stuffy. I've been throwing up a lot too...it's just been a real struggle. I don't know what's going on, but I hope you can help me figure it out!"
disease = predict_disease(text)
print(disease)

Common Cold


## Medicine Prediction

We need to apply LLM here to give a natural prescription mentioneing the patients name, predicted disease and what medicines or tests it s/he can go through. The following dataset doesn't give us a lot of options to chose our desired medicine based on the disease

In [61]:
drugs_df = pd.read_csv("/kaggle/input/drug-dataset/drugs_side_effects_drugs_com.csv")
drugs_df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html


In [62]:
drugs_df.medical_condition.value_counts()

medical_condition
Pain                    264
Colds & Flu             245
Acne                    238
Hypertension            177
Osteoarthritis          129
Hayfever                124
Eczema                  122
AIDS/HIV                109
Diabetes (Type 2)       104
Psoriasis                93
GERD (Heartburn)         77
Pneumonia                72
Angina                   71
Bronchitis               71
Migraine                 61
Insomnia                 60
Constipation             60
Diabetes (Type 1)        57
Osteoporosis             56
ADHD                     55
Depression               51
Seizures                 50
Bipolar Disorder         47
UTI                      46
Asthma                   45
Anxiety                  45
Cholesterol              45
Diarrhea                 38
Covid 19                 34
Rheumatoid Arthritis     33
Alzheimer's              27
Weight Loss              23
COPD                     23
IBD (Bowel)              22
Cancer                   20
Sc

In [63]:
# Addtional dataset downloaded (new_medicine-dataset)
# https://huggingface.co/datasets/MattBastar/medicine

In [64]:
drugs_df_2 = pd.read_csv("/kaggle/input/new-medicine-dataset/Medicine_Details.csv")
drugs_df_2.head()

,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29


In [65]:
drugs_df_2 = drugs_df_2.rename(columns={
    'Medicine Name': 'drug_name',
    'Uses': 'medical_condition',
})

In [66]:
drugs_df_2.shape

(11825, 9)

In [67]:
len(drugs_df_2['medical_condition'].unique())

712

In [68]:
drugs_df_2['medical_condition'][10]

'Treatment of Bacterial infections'

In [69]:
drugs_df_3 = pd.read_csv('/kaggle/input/drug-dataset-2/drugsComTrain_raw.csv')
drugs_df_3.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [70]:
drugs_df_3 = drugs_df_3.rename(columns={
    'drugName': 'drug_name',
    'condition': 'medical_condition',
})

In [71]:
drugs_df_4 = pd.read_csv('/kaggle/input/drugbank/drugbank.tsv', sep='\t')
drugs_df_4.head()

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description
0,DB00001,Lepirudin,biotech,approved,B01AE02,Antithrombins|Fibrinolytic Agents,NaN,NaN,Lepirudin is identical to natural hirudin exce...
1,DB00002,Cetuximab,biotech,approved,L01XC06,Antineoplastic Agents,NaN,NaN,Epidermal growth factor receptor binding FAB. ...
2,DB00003,Dornase alfa,biotech,approved,R05CB13,Enzymes,NaN,NaN,Dornase alfa is a biosynthetic form of human d...
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,Antineoplastic Agents,NaN,NaN,A recombinant DNA-derived cytotoxic protein co...
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,Immunosuppressive Agents,NaN,NaN,Dimeric fusion protein consisting of the extra...


In [72]:
drugs_df_4['description'][20]

'This drug is the synthetic form of natural secretin.  It is prepared using solid phase peptide synthesis.  Secretin is a peptide hormone produced in the S cells of the duodenum. Its main effect is to regulate the pH of the small  intestine’s contents through the control of gastric acid secretion and buffering with bicarbonate. It was the first hormone to be discovered.'

Seems like drugbank dataset will not be of good use because the medical condition is not simply mentioned. The description column is rather about the contents of the drugs

In [73]:
# Select only the drug_name and medical_condition columns from each dataframe
drugs_df_selected = drugs_df[['drug_name', 'medical_condition']]
drugs_df_2_selected = drugs_df_2[['drug_name', 'medical_condition']]
drugs_df_3_selected = drugs_df_3[['drug_name', 'medical_condition']]

# Concatenate the selected dataframes
drug_df_combined = pd.concat([drugs_df_selected, drugs_df_2_selected, drugs_df_3_selected])

In [74]:
drugs_df_selected.shape

(2931, 2)

In [75]:
drug_df_combined.shape

(176053, 2)

In [76]:
drug_df_combined['drug_name'].nunique()

17045

In [79]:
# saving drugs_df_combined 
drug_df_combined.to_csv("drugs_df_combined.csv")

### Trying LLM

In [77]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [78]:
# Load your dataset
df = drug_df_combined
inputs = df.apply(lambda row: tokenizer(row["drug_name"] + ": " + row["medical_condition"], return_tensors="pt"), axis=1)
# Preprocess the text data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(df["drug_name"], df["medical_condition"], return_tensors="pt")


NameError: name 'tokenizer' is not defined

In [ ]:
# Create a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocess the text data
inputs = tokenizer(texts, return_tensors='pt', max_length=512, padding='max_length', truncation=True)

In [ ]:
# Create a custom dataset class
class DrugDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        inputs = {k: v[idx] for k, v in self.inputs.items()}
        label = self.labels[idx]
        return inputs, label



In [ ]:
# Create a dataset instance and data loader
dataset = DrugDataset(inputs, labels)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Load a pre-trained BERT model and configure it for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))

# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification

In [ ]:
label_to_id = {class_name: idx for idx, class_name in enumerate(classes)}
encoded_labels = [label_to_id[label] for label in labels]


In [ ]:
# Fine-tune the model
for epoch in range(5):  # Adjust the number of epochs as needed
    model.train()
    for batch in data_loader:  # Replace with your actual data loader
        inputs, labels = batch
        inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to device
        labels = labels.to(device)  # Move labels to device
        optimizer.zero_grad()
        outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    model.eval()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load pre-trained model and tokenizer
model_name = "gpt2"  # Replace with your desired LLM (e.g., "EleutherAI/gpt-neo-2.7")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Example input
question = "What is the recommended drug for hypertension?"

# Tokenize input
input_ids = tokenizer.encode(question, return_tensors="pt")

# Generate response
with torch.no_grad():
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

print("Response:", response)


In [ ]:
# Fine Tuning Llama 2 with own dataset 
# https://colab.research.google.com/drive/1I2JretY-7qeX4G4bE2SgQzgkB779P326?usp=sharing#scrollTo=Scor9o08VVhQ

We can finetune LLM model with new drug/medicine datasets we have gathered

In [ ]:
# using chatgpt api
# https://adamfard.com/blog/how-to-use-chatgpt-api